In [51]:
import pandas as pd
import math
import glob
from collections import defaultdict
import os # these two packages are good for searching and navigating file systems
import os.path as op

pd.set_option('display.max_columns', None)


user_path = ''

# function to split classification and get rid of _
def split_classification(df):
    df[['domain', 'phyla', 'class', 'order', 'family', 'genus', 'species']] = df.classification.str.split(';', expand=True)
    df['domain'] = df['domain'].str.replace('d__', '') # remove the d__ in front of all observations
    df['phyla'] = df['phyla'].str.replace('p__', '')
    df['class'] = df['class'].str.replace('c__', '')
    df['order'] = df['order'].str.replace('o__', '')
    df['family'] = df['family'].str.replace('f__', '')
    df['genus'] = df['genus'].str.replace('g__', '')
    df['species'] = df['species'].str.replace('s__', '')
    return df

In [54]:
# for loop: create meta data columns for result data frame
# if data_type = SAG use sag lookup table
# if data_type = MAG use mag lookup table

result = pd.read_csv('/Users/juliabrown/Google Drive/My Drive/projects/OMZvir_round2/MH_project/dramv_full_summary.csv')

sag_cols = ['vir_id','depth', 'classification_via_GTDBTk']
sag_tax = pd.read_csv('/Users/juliabrown/Google Drive/My Drive/projects/OMZvir_round2/MH_project/sag_data/MPvsag_info_230818.csv')[sag_cols]
sag_tax.rename(columns={'classification_via_GTDBTk': 'classification','depth':'sample_depth'}, inplace=True)
sag_tax = split_classification(sag_tax)
sag_tax['virus_name'] = sag_tax['vir_id'].str.replace('vir', 'cv1')

mag_tax = pd.read_csv('/Users/juliabrown/Google Drive/My Drive/projects/OMZvir_round2/MH_project/tables/vMAG_associations.csv')
mag_cols= ['virus_name', 'sample_depth','domain', 'phyla', 'class', 'order', 'family', 'genus', 'species']

def rename_vmag(virid, sample_name):
    new_samp = '{}-{}'.format(sample_name[:2].lower(), sample_name[2:])
    return '{}-{}'.format(new_samp, virid)

mag_tax['virus_name'] = [rename_vmag(l['virus_name'],l['sample_name']) for i, l in mag_tax.iterrows()]

'''
# above list comprehension is the same as this code:

new_list = []

for i, l in mag_tax.iterrows():
    new_list.append(rename_vmag(l['virus_name'],l['sample_name']))
    
mag_tax['virus_name'] = new_list
'''
mag_tax = mag_tax[mag_cols]

In [55]:
tax_df = pd.concat([sag_tax,mag_tax])[mag_tax.columns]
tax_df.to_csv('/Users/juliabrown/Google Drive/My Drive/projects/OMZvir_round2/MH_project/tables/091923_all_vir_host_taxonomy.csv',
              index = False)

In [56]:
tax_df

,virus_name,sample_depth,domain,phyla,class,order,family,genus,species
0,cv1_AM-654-B02,80,Bacteria,Proteobacteria,Gammaproteobacteria,Arenicellales,UBA5680,UBA5680,UBA5680 sp002420425
1,cv1_AM-654-B17,80,Bacteria,Proteobacteria,Alphaproteobacteria,Pelagibacterales,Pelagibacteraceae,Pelagibacter,
2,cv1_AM-654-B06,80,Bacteria,Proteobacteria,Alphaproteobacteria,Pelagibacterales,Pelagibacteraceae,GCA-2704625,GCA-2704625 sp017640245
3,cv1_AM-654-C02,80,Bacteria,Proteobacteria,Alphaproteobacteria,Pelagibacterales,Pelagibacteraceae,Pelagibacter,
4,cv1_AM-654-B04,80,Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales,BACL11,DUAL01,
...,...,...,...,...,...,...,...,...,...
127,jv-154-vMAG_44,140,Bacteria,Planctomycetota,Planctomycetia,Pirellulales,Pirellulaceae,ARS98,ARS98 sp002685655
128,jv-154-vMAG_17,140,Bacteria,Armatimonadota,UBA5377,UBA5377,VGFC01,NaN,NaN
129,jv-154-vMAG_17,140,Bacteria,Armatimonadota,UBA5377,UBA5377,VGFC01,NaN,NaN
130,jv-154-vMAG_17,140,Bacteria,Armatimonadota,UBA5377,UBA5377,VGFC01,NaN,NaN


In [58]:
result.merge(tax_df, left_on = 'ID', right_on = 'virus_name', how = 'outer')

,ID,pfam_count,kegg_count,viral_hit,vogdb_count,total_genes_annot,V_count,M_count,A_count,P_count,E_count,K_count,T_count,F_count,B_count,virus_name,sample_depth,domain,phyla,class,order,family,genus,species
0,jv-132-vMAG_16,3.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,39.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,jv-132-vMAG_3,4.0,0.0,0.0,0.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cv1_AM-664-O20,7.0,0.0,0.0,0.0,7.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,34.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cv1_AM-680-O21,5.0,4.0,0.0,0.0,9.0,1.0,3.0,0.0,0.0,0.0,2.0,0.0,15.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cv1_AM-660-G21,10.0,4.0,0.0,3.0,17.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,29.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,jv-121-vMAG_25,95.0,Unclassified,NaN,NaN,NaN,NaN,NaN,NaN
421,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,jv-121-vMAG_16,95.0,Bacteria,Planctomycetota,Planctomycetia,Pirellulales,UBA7805,UBA7805,NaN
422,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,jv-121-vMAG_16,95.0,Bacteria,Planctomycetota,Planctomycetia,Pirellulales,UBA7805,UBA7805,NaN
423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,jv-121-vMAG_16,95.0,Bacteria,Planctomycetota,Planctomycetia,Pirellulales,UBA7805,UBA7805,NaN


In [28]:
result[result['ID'].str.contains('jv')]

,ID,pfam_count,kegg_count,viral_hit,vogdb_count,total_genes_annot,V_count,M_count,A_count,P_count,E_count,K_count,T_count,F_count,B_count
0,jv-132-vMAG_16,3,0,0,0,3,1,0,0,0,0,0,0,39,0
1,jv-132-vMAG_3,4,0,0,0,4,2,0,0,0,0,0,0,13,0
6,jv-119-vMAG_13,6,2,0,7,15,0,0,0,0,0,0,0,30,0
7,jv-121-vMAG_62,4,1,0,2,7,2,0,0,0,0,0,0,27,0
8,jv-121-vMAG_76,5,2,0,1,8,0,1,0,0,0,0,0,22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,jv-154-vMAG_42,9,0,0,7,16,9,0,0,1,0,0,0,25,0
326,jv-119-vMAG_36,3,0,0,4,7,1,0,0,0,0,0,0,16,0
327,jv-119-vMAG_22,4,0,0,4,8,3,0,0,0,0,0,0,23,0
329,jv-132-vMAG_33,7,3,0,4,14,2,0,0,0,0,0,0,49,0


In [23]:
tax_df['virus_name']

0      cv1_AM-654-B02
1      cv1_AM-654-B17
2      cv1_AM-654-B06
3      cv1_AM-654-C02
4      cv1_AM-654-B04
            ...      
127           vMAG_44
128           vMAG_17
129           vMAG_17
130           vMAG_17
131           vMAG_17
Name: virus_name, Length: 176, dtype: object

In [ ]:
for index, row in df.iterrows():
    if result['data_type'] == 'vSAG':
        result.merge(sag_tax, on=['ID','virus_name'], how='left')
    else: result.merge(mag_tax, on=['ID','vir_id'], how = 'left')